In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.utils import to_categorical
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
num_classes = 10

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Resize the images to the VGG16 input size (32x32x3)
def resize_images(images, size):
    resized_images = []
    for img in images:
        resized_img = np.resize(img, size)
        resized_images.append(resized_img)
    return np.array(resized_images)

X_train = resize_images(X_train, (32, 32))
X_test = resize_images(X_test, (32, 32))

# Add a channel for grayscale (3rd dimension) images
X_train = np.stack((X_train,) * 3, axis=-1)
X_test = np.stack((X_test,) * 3, axis=-1)

# Normalize the images
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# Convert labels to one-hot encoding format
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


In [9]:
conv_base = VGG16(weights = None, include_top=False, input_shape=(32, 32, 3))

conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == "block5_conv1":
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

optimizer = keras.optimizers.legacy.Adam(learning_rate=0.0002) # Original is 0.0001

model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])

In [10]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [12]:
earlyStopping = EarlyStopping(monitor = 'val_loss', patience = 3, verbose = 0, mode = 'min') 
mcp_save = ModelCheckpoint('best_weights.hdf5', save_best_only = True, monitor = 'val_loss', mode = 'min') 
reduce_lr_loss = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.4, patience = 7, verbose = 1, min_delta = 1e-4, mode = 'auto') # Reduz o learning_rate quando a métrica de avaliação para de melhorar

history = model.fit(datagen.flow(X_train, y_train, batch_size=64), epochs = 10, validation_data = (X_test, y_test), callbacks = [earlyStopping, mcp_save, reduce_lr_loss])

Epoch 1/10
938/938 [==============================] - 279s 298ms/step - loss: 1.2155 - accuracy: 0.5268 - val_loss: 1.1930 - val_accuracy: 0.5262 - lr: 2.0000e-04
Epoch 2/10
938/938 [==============================] - 301s 321ms/step - loss: 1.1665 - accuracy: 0.5474 - val_loss: 1.2248 - val_accuracy: 0.5380 - lr: 2.0000e-04
Epoch 3/10
938/938 [==============================] - 312s 332ms/step - loss: 1.1445 - accuracy: 0.5566 - val_loss: 1.1046 - val_accuracy: 0.5859 - lr: 2.0000e-04
Epoch 4/10
938/938 [==============================] - 317s 338ms/step - loss: 1.1185 - accuracy: 0.5701 - val_loss: 1.0775 - val_accuracy: 0.5962 - lr: 2.0000e-04
Epoch 5/10
938/938 [==============================] - 321s 342ms/step - loss: 1.0904 - accuracy: 0.5821 - val_loss: 0.9886 - val_accuracy: 0.6272 - lr: 2.0000e-04
Epoch 6/10
938/938 [==============================] - 330s 351ms/step - loss: 1.0711 - accuracy: 0.5909 - val_loss: 1.0209 - val_accuracy: 0.6106 - lr: 2.0000e-04
Epoch 7/10
938/938 [==

In [13]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

class_names = ['Top', 'Trousers', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
report = classification_report(y_test_classes, y_pred_classes, target_names=class_names)
print(report)

313/313 [==============================] - 26s 82ms/step
              precision    recall  f1-score   support

         Top       0.57      0.59      0.58      1000
    Trousers       0.55      0.93      0.69      1000
    Pullover       0.53      0.20      0.30      1000
       Dress       0.46      0.59      0.52      1000
        Coat       0.43      0.67      0.52      1000
      Sandal       0.64      0.82      0.72      1000
       Shirt       0.42      0.08      0.14      1000
     Sneaker       0.83      0.65      0.73      1000
         Bag       0.94      0.74      0.83      1000
  Ankle boot       0.80      0.80      0.80      1000

    accuracy                           0.61     10000
   macro avg       0.62      0.61      0.58     10000
weighted avg       0.62      0.61      0.58     10000

